In [1]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
import numpy as np

from rdkit.Chem import MACCSkeys
import pickle
from exmol_our import exmol
from functools import partial

from sklearn.ensemble import RandomForestRegressor
import pickle
import pandas as pd
from sklearn.model_selection import KFold
import random

import warnings
warnings.filterwarnings('ignore')
import json
import pickle
import scipy.cluster.hierarchy as hc

from rdkit import Chem  
from rdkit.Chem import MolFromSmiles as smi2mol 

from rdkit.Chem import AllChem  
from rdkit.DataStructs.cDataStructs import BulkTanimotoSimilarity, TanimotoSimilarity 
from rdkit.Chem import Mol  

import warnings
warnings.simplefilter("ignore", UserWarning)

In [2]:
data_type = 'protein'
feature_type = 'maccs'
model_type = 'lgbm'

## Load fingerprint data

In [3]:
data_path = f'../../data/processed/{data_type}_{feature_type}_all.csv'
df = pd.read_csv(data_path)

# 2. Split data to X and y 
X_fp = df.loc[:, df.columns != df.columns[0]]
y_fp = df[df.columns[0]]

X_fp = np.array(X_fp)
y_fp = np.array(y_fp)

## Get predictions

In [4]:
first_splits = KFold(n_splits=5, shuffle=True, random_state=42)
predictions = np.zeros((len(X_fp)))
for fold_1, (train_idx,test_idx) in enumerate(first_splits.split(np.arange(len(X_fp)))):
    train_x = X_fp[train_idx]
    test_x = X_fp[test_idx]
    model = pickle.load(open(f'../../models/{data_type}_{feature_type}_{model_type}_{fold_1+1}.pkl', 'rb'))
    predictions[test_idx] = model.predict(test_x)

## Load smiles data

In [5]:
data_path = f'../../data/processed/{data_type}_smiles_all.csv'
df = pd.read_csv(data_path)
df['predictions'] = predictions
df = df[df['predictions']>85] # get inactive examples

X_sm = df[['smiles']]
y_sm = df[['protein_plasma_binding']]

In [6]:
len(X_sm)

1813

## Clustering

In [7]:
ECFP4 = [AllChem.GetMorganFingerprint(Chem.MolFromSmiles(smiles[0]), 2) for smiles in X_sm.values]
M = np.array([BulkTanimotoSimilarity(f, ECFP4) for f in ECFP4])
M2 = 1 - M
dist_df = pd.DataFrame(M2, index = X_sm.values, columns= X_sm.values)
clustered = hc.linkage(M2, method='complete')

In [8]:
from scipy.cluster.hierarchy import cut_tree
n_clusters = 20
clusters = cut_tree(clustered, n_clusters = n_clusters).T
df_clustered = pd.DataFrame({'Column1': list(X_sm.values), 'Column2': clusters[0]})

In [9]:
from collections import Counter
c = Counter(list(df_clustered['Column2']))
sorted(c.items(), key=lambda i: i[1])

[(17, 17),
 (18, 21),
 (6, 24),
 (19, 30),
 (16, 34),
 (15, 37),
 (14, 40),
 (13, 42),
 (12, 46),
 (9, 49),
 (11, 51),
 (4, 71),
 (7, 80),
 (3, 81),
 (10, 81),
 (8, 144),
 (1, 184),
 (0, 220),
 (5, 264),
 (2, 297)]

## Get samples

In [10]:
random.seed(42)
np.random.seed(42)
n_clusters = 100
x = 1000/len(X_sm.values)
samples = []
for i in range(n_clusters):
    C_i = np.where(df_clustered.Column2 == i)[0].tolist() 
    n_i = len(C_i)
    sample_i = random.sample(C_i, round(x * n_i)) 
    samples += list(sample_i)
print(len(samples))

999


## Exmol

In [12]:
def GetMACCS(smiles):
    """Function to convert smiles to numpy array of MACCS keys"""
    mol = Chem.MolFromSmiles(smiles)
    fp = MACCSkeys.GenMACCSKeys(mol)
    fp.ToBitString() 
    fp_list = list(fp)[1:]
    return np.array(fp_list)

In [13]:
def model_predict(smiles, model):
    fingerprint = GetMACCS(smiles)
    prediction  = model.predict(fingerprint.reshape(1,-1))
    return prediction

In [14]:
def get_changed_idxs(cfs):
    orignal_smiles = cfs[0].smiles
    orignal_fs = GetMACCS(orignal_smiles)
    counterfc_fp = [GetMACCS(counterfc.smiles) for counterfc in cfs[1:] if counterfc.label.startswith('Increase')]
    return [(np.where((orignal_fs-cfp)==1)[0], np.where((orignal_fs-cfp)==-1)[0]) for cfp in counterfc_fp] # (-,+)

In [15]:
def get_cfs(examples, model, thd):
    changed_indx_list = []
    for exp in examples.values:
        space = exmol.sample_space(exp[0], partial(model_predict, model=model), preset="medium", num_samples=500, batched=False)
        prediction = model_predict(exp[0], model)[0]
        if prediction<0:
            radius = round(float(thd+abs(prediction)),2)
        else:
            radius = round(float(abs(thd-prediction)),2)
        cfs = exmol.rcf_explain(space, radius, nmols=4)
        changed_idxs = get_changed_idxs(cfs)
        changed_indx_list += changed_idxs
    return changed_indx_list 

In [16]:
first_splits = KFold(n_splits=5, shuffle=True, random_state=42)
changed_indx_list = []
for fold_1, (train_idx,test_idx) in enumerate(first_splits.split(np.arange(len(X_sm)))):
    test_idx = np.array(list(set(test_idx) & set(samples)))
    test_x = X_sm.iloc[test_idx]
    model = pickle.load(open(f'../../models/{data_type}_{feature_type}_{model_type}_{fold_1+1}.pkl', 'rb'))
    changed_indx_list += get_cfs(test_x, model, 85)

🤘Done🤘: 100%|██████████| 85.0/85 [00:00<00:00, 1658.61it/s]                      
🤘Done🤘: 100%|██████████| 98.0/98 [00:00<00:00, 1410.83it/s]                      
🤘Done🤘: 100%|██████████| 84.0/84 [00:00<00:00, 1901.73it/s]                      
🤘Done🤘: 100%|██████████| 183.0/183 [00:00<00:00, 1086.64it/s]                    
🤘Done🤘: 100%|██████████| 180.0/180 [00:00<00:00, 1125.24it/s]                    
🤘Done🤘: 100%|██████████| 112.0/112 [00:00<00:00, 1569.01it/s]                    
🤘Done🤘: 100%|██████████| 210.0/210 [00:00<00:00, 969.39it/s]                     
🤘Done🤘: 100%|██████████| 79.0/79 [00:00<00:00, 2046.33it/s]                      
🤘Done🤘: 100%|██████████| 127.0/127 [00:00<00:00, 1063.31it/s]                    
🤘Done🤘: 100%|██████████| 162.0/162 [00:00<00:00, 1539.15it/s]                    
🤘Done🤘: 100%|██████████| 167.0/167 [00:00<00:00, 1331.49it/s]                    
🤘Done🤘: 100%|██████████| 100.0/100 [00:00<00:00, 1688.57it/s]                    
🤘Done🤘: 100%|███

In [17]:
len(changed_indx_list)

975

#### Mean removed fingerprints

In [18]:
np.mean([len(list(c[0])) for c in changed_indx_list])

4.449230769230769

#### Mean added fingerprints

In [19]:
np.mean([len(list(c[1])) for c in changed_indx_list])

5.123076923076923

#### Most frequent removed fingerprints

In [20]:
with open('../../maccs_keys_dict.pickle', 'rb') as fp:
    Maccs_keys_dict = pickle.load(fp)

In [21]:
def map_to_smarts(df):
    df_mapped = df + 1
    df_mapped = df_mapped.applymap(lambda x: Maccs_keys_dict[x][0])
    return df_mapped

In [22]:
from collections import Counter
counts = Counter([item for sublist in [list(c[0]) for c in changed_indx_list] for item in sublist])
print(len(counts))
sorted_counts = sorted(counts.items(), key=lambda i: i[1], reverse=True)[:10]
df_sorted_counts = pd.DataFrame({'Smarts': [x[0] for x in sorted_counts], 'Counts': [x[1] for x in sorted_counts]})
df_sorted_counts

139


,Smarts,Counts
0,84,94
1,74,86
2,112,81
3,135,80
4,96,77
5,92,76
6,131,71
7,151,70
8,85,67
9,137,65


In [23]:
df_sorted_counts['Smarts'] = map_to_smarts(df_sorted_counts[['Smarts']])
df_sorted_counts.to_csv(f'files/{data_type}/{data_type}_{feature_type}_{model_type}_most_frequent_removed_fps.csv', index=False)
df_sorted_counts

,Smarts,Counts
0,[#6]~[#7](~[#6])~[#6],94
1,*!@[#7]@*,86
2,[#8]!:*:*,81
3,[#8]=*,80
4,[#7]~*~*~*~[#8],77
5,[!#6;!#1]~[CH3],76
6,[#8]~*~[CH2]~*,71
7,[#8]~[#6](~[#6])~[#6],70
8,"[C;H2,H3][!#6;!#1][C;H2,H3]",67
9,[!#6;!#1]~[CH2]~*,65


#### Most frequent added fingerprints

In [24]:
counts = Counter([item for sublist in [list(c[1]) for c in changed_indx_list] for item in sublist])
print(len(counts))
sorted_counts = sorted(counts.items(), key=lambda i: i[1], reverse=True)[:10]
df_sorted_counts = pd.DataFrame({'Smarts': [x[0] for x in sorted_counts], 'Counts': [x[1] for x in sorted_counts]})
df_sorted_counts

144


,Smarts,Counts
0,98,247
1,75,182
2,49,152
3,87,121
4,80,117
5,44,112
6,90,99
7,68,94
8,130,89
9,123,84


In [25]:
df_sorted_counts['Smarts'] = map_to_smarts(df_sorted_counts[['Smarts']])
df_sorted_counts.to_csv(f'files/{data_type}/{data_type}_{feature_type}_{model_type}_most_frequent_added_fps.csv', index=False)
df_sorted_counts

,Smarts,Counts
0,[#6]=[#6],247
1,[#6]=[#6](~*)~*,182
2,[#6]=[#6](~[#6])~[#6],152
3,[#16],121
4,[#16]~*(~*)~*,117
5,[#6]=[#6]~[#7],112
6,"[$([!#6;!#1;!H0]~*~*~*~[CH2]~*),$([!#6;!#1;!H0...",99
7,[!#6;!#1]~[!#6;!#1;!H0],94
8,[!#6;!#1;!H0],89
9,[!#6;!#1]~[!#6;!#1],84
